# Maximum likelihood estimation of the Bivariate Poisson distribution

In [244]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import collections.abc
factorial = np.vectorize(math.factorial)
combination = lambda x,y: factorial(x)/(factorial(y)*factorial(x-y))

## Example data

In [245]:
X = np.random.randint(0,10,100)
Y = np.random.randint(0,10,100)

## Implementation

The probabilty function of $(X,Y) \sim BP(\lambda_1, \lambda_2, \lambda_3)$  is
$$
    p(x,y) = P(X = x, Y = y) = e^{-(\lambda_1+\lambda_2+\lambda_3)} \frac{\lambda_1^x}{x!}\frac{\lambda_2^y}{y!} \sum_{i = 0}^{min(x,y)}{\binom xi \binom yi i! \left(\frac{\lambda_3}{\lambda_1\lambda_2}\right)^i}
$$

Then the log of the probabilty function is 
$$
    \log(p(x, y) ) = -(\lambda_1+\lambda_2+\lambda_3) + x \log({\lambda_1}) - \log(x!) +  y \log(\lambda_2) - \log(y!) + \log \left( \sum_{i = 0}^{min(x,y)}{\binom xi \binom yi i! \left(\frac{\lambda_3}{\lambda_1\lambda_2}\right)^i} \right)
$$

In [246]:
def dbp(x, y, l1, l2, l3, logged = True):
    """ Computes the density function"""
    n = 0
    if isinstance(x, (collections.abc.Sequence, np.ndarray)):
        if(x.shape[0] == y.shape[0]): n = len(x)
    suma = - (l1 + l2 + l3)
    suma += x* np.log(l1) - np.log(factorial(x))
    suma += y* np.log(l2) - np.log(factorial(y))
    sumas = np.array([])
    for i in range(n):
        z = min(x[i], y[i])
        if(z != 0):
            I = np.array(range(z+1)) # define I = [0,1,..,min(x,y)]
            sumas = np.append(sumas, np.log( (combination(x[i],I) * combination(y[i],I)*factorial(I)*np.power(l3/(l2*l1),I)).sum()))
        else: sumas = np.append(sumas, [0])
    if(n == 0): 
        I = np.array(range(min(x,y)+1))
        suma += np.log( (combination(x,I) * combination(y,I)*factorial(I)*np.power(l3/(l2*l1),I)).sum())
    else: suma += sumas
    return suma if logged else np.exp(suma)

In [247]:
dbp(X,Y,1,2,3)

array([ -5.58337895,  -3.81650733,  -4.61093171,  -6.        ,
        -3.33736049,  -7.30527533,  -9.07412291, -18.80182748,
        -4.70172763,  -3.75771551,  -4.6533709 ,  -4.70172763,
        -3.33736049,  -9.17805383,  -5.50582009,  -8.42036813,
        -3.75771551,  -4.03050768,  -5.09584135,  -3.13383872,
        -9.88935421,  -7.30527533,  -3.74374975,  -4.70172763,
        -4.53268661,  -3.69187473, -11.38966715,  -3.90253405,
        -6.        ,  -4.88142074,  -3.33736049,  -7.79175947,
        -5.05791261, -12.57925121,  -3.3336724 ,  -6.40546511,
        -5.18168968,  -7.30527533,  -5.90201633,  -3.94161187,
        -3.24846469,  -4.39056209,  -3.92055846,  -6.2677151 ,
       -16.6046029 ,  -6.        ,  -7.47431635,  -4.61370564,
        -6.        ,  -5.09584135,  -3.55765296,  -6.35312871,
        -3.75771551, -10.80506142,  -3.75771551,  -5.9834707 ,
        -9.07412291,  -5.3938642 ,  -3.13383872,  -2.99872755,
        -5.0454886 ,  -7.9542784 , -13.34650636, -12.57

Given $(X_1,Y_2),\dots,(X_n,Y_n)$ Bivariate Poisson random variables with parameters $(\lambda_1,\lambda_2, \lambda_3)$ then the log likelihood function is 

$$
    \text{llf} = \log(\prod_{i=0}^{n}{p(x_{i},y_{i})}) = \sum_{i= 0}^{n}{\log(p(x_{i},y_{i}))} 
$$

$$
    = -n(\lambda_1+\lambda_2+\lambda_3) + \sum_{i=0}^{n}{\left( x_i \log(\lambda_1) - \log(x_{i}!) \right)} + \sum_{i=0}^{n}{\left( y_i \log(\lambda_2) - \log(y_{i}!) \right)} + \sum_{j=1}^{n}{\log\left( \sum_{i = 0}^{min(x_j,y_j)}{\binom {x_j}i \binom {y_j}i i! \left(\frac{\lambda_3}{\lambda_1\lambda_2}\right)^i} \right)}
$$


In [248]:
def llf(X,Y,l1,l2,l3):
    """ Computes the log likelihood function of BP(l1,l2,l3)"""
    
    if X.shape[0] != Y.shape[0]: raise Exception("X and Y must have the same length") 
    n = X.shape[0]
    suma = - n * (l1 + l2 + l3)
    suma += ( X * np.log(l1) - np.log(factorial(X)) ).sum()
    suma += ( Y * np.log(l2) - np.log(factorial(Y)) ).sum()
    for i in range(n):
        z = min(X[i],Y[i])
        if z == 0: continue
        I = np.array(range(z+1))
        suma += np.log((((factorial(X[i])*factorial(Y[i]))/(factorial(X[i] - I)*factorial(I)*factorial(Y[i] - I))) * np.power(l3/(l1*l2),I)).sum())
    return suma 

## Algorithm to get $\hat{\lambda}_1$, $\hat{\lambda}_2$, $\hat{\lambda}_3$ of ML

From [Kawamura 1984] we have that $\hat{\lambda}_1 + \hat{\lambda}_3 = \hat{X}$ and $\hat{\lambda}_2 + \hat{\lambda}_3 = \hat{Y}$ with $\hat{\lambda}_3 \in{[0,\min(\hat{X},\hat{Y})]}$ 

In [249]:
def max_llf_l3(X,Y, iterations, size_step = 10):
    """ Algorthm to find the l3 that maximize the log likelihood function of BP(l1,l2,l3)"""

    x_hat = X.sum()/X.shape[0]
    y_hat = Y.sum()/Y.shape[0]
    interval = (0,min(x_hat,y_hat))
    d = np.linspace(interval[0],interval[1],size_step)

    print(interval)
    for i in range(iterations):
        lff_d = np.array([ llf(X,Y,X.sum()/X.shape[0]-l,Y.sum()/Y.shape[0]-l,l) for l in d[0:size_step-2]]) # [0,min(x_hat,y_hat))
        print("llf values", lff_d)
        indexes = np.where(lff_d == lff_d.max()) # Maximizing lff on D_i
        print("Minimum values", indexes)
        index = indexes[0][0]
        l3 = d[index]
        if(index == 0 or index == 9): 
            if(index == 0): 
                interval = (d[index], d[index+1])
            else: 
                interval = (d[index-1], d[index])
        else: interval = (d[index - 1], d[index + 1])
        print("D_{} = ".format(i), interval)
        d = np.linspace(interval[0],interval[1],size_step)
    
    return l3

In [250]:
lambda3 = max_llf_l3(X,Y, iterations=10)

(0, 4.41)
llf values [-502.4180624  -503.01659117 -506.38240759 -513.26256338 -525.12031237
 -544.70165066 -577.51552272 -636.36454388]
Minimum values (array([0]),)
D_0 =  (0.0, 0.49)


llf values [-502.4180624  -502.35894573 -502.33085147 -502.33381205 -502.36795201
 -502.43348282 -502.53069888 -502.65997461]
Minimum values (array([2]),)
D_1 =  (0.05444444444444444, 0.16333333333333333)
llf values [-502.35894573 -502.35002196 -502.34262973 -502.33676924 -502.33244089
 -502.32964534 -502.32838348 -502.32865639]
Minimum values (array([6]),)
D_2 =  (0.11493827160493827, 0.1391358024691358)
llf values [-502.32964534 -502.32923235 -502.32889511 -502.32863363 -502.32844792
 -502.32833799 -502.32830386 -502.32834554]
Minimum values (array([6]),)
D_3 =  (0.12838134430727022, 0.13375857338820302)
llf values [-502.32833799 -502.32832385 -502.32831346 -502.32830681 -502.32830391
 -502.32830475 -502.32830933 -502.32831765]
Minimum values (array([4]),)
D_4 =  (0.1301737540009145, 0.13136869379667732)
llf values [-502.32830681 -502.32830584 -502.32830506 -502.32830446 -502.32830404
 -502.32830381 -502.32830377 -502.32830391]
Minimum values (array([6]),)
D_5 =  (0.13083760944300496

In [251]:
# The paramters of MLE are 
lambda1 = np.mean(X) - lambda3
lambda2 = np.mean(y) - lambda3
lambda1, lambda2, lambda3

(4.439063798765687, 5.869063798765687, 0.13093620123431313)

# References


Kawamura, Kazutomo: Direct calculation of maximum
likelihood estimator for the bivariate Poisson distribution. In: Kodai Mathe-
matical Journal 7 (1984), Nr. 2, S. 211 – 221. – URL https://doi.org/10.
2996/kmj/1138036908